In [24]:
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
fashion_mnist_train = torchvision.datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor()) # normalized
fashion_mnist_test = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor()) # normalized

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model =  nn.Sequential(nn.Conv2d(1,6,5, padding=2),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(2, stride=2),
                                    nn.Conv2d(6,16,5),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(2, stride=2),
                                    nn.Flatten(),
                                    nn.Linear(400, 120),
                                    nn.Sigmoid(),
                                    nn.Linear(120, 84),
                                    nn.Sigmoid(),
                                    nn.Linear(84, 10))

    def forward(self, x):
        x = self.model(x)
        return x

In [63]:
lenet = LeNet()
lenet

LeNet(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [49]:
epochs = 10
learning_rate = 0.01
batch_size = 12

In [ ]:
dataloader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
loss = None
optimizer = None

In [41]:
for train_features, train_labels in iter(dataloader):
    continue

C:\Users\abhin\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
